In [ ]:
# New Notebook Starts Here 

import csv
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import time
import datetime

def load_price_data(path_to_data_dir="../data/",price_file="eth_usd_price"):
    with open(path_to_data_dir+price_file+".csv","r") as infile:
        try:
            df = pd.read_csv(infile)
            print(df.shape)
        except Exception as e:
            print(e)

    df['datetime'] = pd.to_datetime(df['unix_timestamp'], unit='s')    
    df.sort_values(by="unix_timestamp", axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')
    df['date'] = df['datetime'].dt.date
    return(df)

def process_daily_summary(price_data):
    price_data["weighted_avg"] = price_data["volume"]*price_data["close"]    
    price_data['volatility'] = price_data['close'].pct_change()
    
    daily_price_data = price_data.groupby("date").agg({
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last",
        "volume": "sum",
        "unix_timestamp": "first",
        "weighted_avg": "sum",
        "volatility": "sum",
        })
    
    daily_price_data["weighted_avg"] = daily_price_data["weighted_avg"]/daily_price_data["volume"]
    return(daily_price_data)

def price_analysis(daily_price_data,price_file = "usdt_usdc_price",
                   jump_threshold=.04,recovery_mean=None,recovery_threshold=0.005,
                   trunc_window_start=None,trunc_window_end=None):        
    #### Compute GBM Mu and Sigma
    daily_price_data['daily_return'] = daily_price_data['weighted_avg'].pct_change()
    daily_price_data['log_daily_return'] = np.log(daily_price_data['weighted_avg'] / daily_price_data['weighted_avg'].shift(1))

    # Drop NaN values (the first entry will be NaN due to the shift)
    daily_price_data = daily_price_data.dropna()

    # Calculate mean (mu) and standard deviation (sigma) of daily returns
    mu = daily_price_data['log_daily_return'].mean()
    sigma = daily_price_data['log_daily_return'].std()

    print("Mean (mu) of daily returns:", mu)
    print("Standard deviation (sigma) of daily returns:", sigma)

    plt.figure()
    plt.plot(daily_price_data["weighted_avg"])

    #### Jump Diffusion - Magnitude Analysis
    daily_price_data["drops"]=(daily_price_data["high"]-daily_price_data["low"])/daily_price_data["weighted_avg"]
    counts,bin_edges = np.histogram(daily_price_data["drops"],bins=100)
    hist_daily_df = pd.DataFrame({"Bin_Left_Edge":bin_edges[:-1],"Bin_Reft_Edge":bin_edges[1:],"Count":counts})
    plt.figure()
    plt.hist(daily_price_data["drops"],bins=100,log=True)
    plt.show()

    #### Jump Diffusion - Recovery Analysis     
    recovery_times = []
    jump_state = False
    jump_timer=0    

    daily_price_data["jump_state"]=False
    daily_price_data["recovery_mean"]=recovery_mean    
    daily_price_data["ema"]=daily_price_data["weighted_avg"].ewm(span=9, adjust=False).mean()

    for index, row in daily_price_data.iterrows():    
        jump_clear = abs(row["daily_return"])>=jump_threshold
        
        if recovery_mean==None:            
            daily_price_data.loc[index,"recovery_mean"] = daily_price_data.loc[index,"ema"]
        in_recovery_range = abs(daily_price_data.loc[index,"recovery_mean"]-row["weighted_avg"])<daily_price_data.loc[index,"recovery_mean"]*recovery_threshold
        
        if jump_state==True:
            jump_timer+=1
            if in_recovery_range:
                recovery_times+=[jump_timer]
                jump_state=False
                jump_timer=0    
            else:
                pass
        else: 
            if jump_clear:
                jump_state=True
            else:
                pass    
        daily_price_data.loc[index, "jump_state"] = jump_state

    plt.figure()
    plt.plot(daily_price_data["daily_return"])
    plt.axhline(y=jump_threshold, color='r', linestyle='--')
    plt.axhline(y=-1*jump_threshold, color='r', linestyle='--')

    daily_price_data.index = pd.to_datetime(daily_price_data.index)
    
    if trunc_window_start!=None and trunc_window_end!=None:
        trunc_daily_price_data = daily_price_data[(daily_price_data.index>=trunc_window_start) & (daily_price_data.index<=trunc_window_end)]
    else:
        trunc_daily_price_data = daily_price_data
    plt.figure()
    plt.plot(trunc_daily_price_data["weighted_avg"],color = '#260099')
    plt.plot(trunc_daily_price_data["recovery_mean"], color='k', linestyle='--')
    plt.plot(trunc_daily_price_data["recovery_mean"]*(1+recovery_threshold), color='g', linestyle='--')
    plt.plot(trunc_daily_price_data["recovery_mean"]*(1-recovery_threshold), color='g', linestyle='--')
    color= 'green'
    for i, row in trunc_daily_price_data.iterrows():
        left_edge = i - pd.to_timedelta(.5, unit='D')
        right_edge = i + pd.to_timedelta(.5, unit='D')
        if row['jump_state']==True: plt.axvspan(left_edge, right_edge, color="red", alpha=0.2)
        else: plt.axvspan(left_edge, right_edge, color="green", alpha=0.1)    
    plt.xticks(rotation=45)

In [ ]:
#### Stablecoin Price Analysis
price_file="usdt_usdc_price"
price_data = load_price_data(path_to_data_dir="../data/",price_file=price_file)
daily_price_data = process_daily_summary(price_data)
price_analysis(daily_price_data,price_file=price_file,
               jump_threshold = .04,recovery_mean = 1,recovery_threshold = 0.01,
               trunc_window_start=datetime.datetime(2023, 3, 1),
               trunc_window_end=datetime.datetime(2023, 3, 30))

In [ ]:
#### ETH Price Analysis
price_file="eth_usd_price"
price_data = load_price_data(path_to_data_dir="../data/",price_file=price_file)
daily_price_data = process_daily_summary(price_data)
price_analysis(daily_price_data,price_file=price_file,
               jump_threshold = .1,recovery_mean = None,recovery_threshold = 0.05,
               trunc_window_start=None,
               trunc_window_end=None)